## Goal of this notebook

The goal of this notebook is to optimize the matching process between the detector network and the "main roads" of OpenStreetMap. It is a refined version of the notebook match_osm_networks.ipynb.

This notebook matches the links of the Paris counting network with the links of an OSM network:
- Load [OpenStreetMap network](https://www.openstreetmap.org/#map=7/51.330/10.453) in Bvd Peripherique
- Load [detector network](https://opendata.paris.fr/explore/dataset/referentiel-comptages-routiers/information/)
- Perform matching by angle and centroid


In [9]:
import numpy as np
import numpy.linalg as la
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm

import osmnx as ox
import momepy
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import alphashape
from pyproj import Proj, Geod
import ast
import matplotlib.font_manager as font_manager
import datetime as dt

from functions import line_length_in_meters, parse_and_average_lanes, approximate_number_of_lanes 

fontsize = 20
figsize = (15, 10)
font = 'Times New Roman'

data_path = '../../data/'  
polygon_road_network = gpd.read_file(data_path + 'network/QGIS_Project/referentiel-comptages-edit.shp')
paris_districts = gpd.read_file(data_path + 'districts_paris.geojson')
df_car_detectors = gpd.read_file(data_path + 'all_car_detectors.geojson')

## Load networks

In [10]:
# perform it for years 2013 - 2024. One cannot retrieve detector data from OSM from before 2013.
year = 2024

with_boulevard_peripherique = True

if with_boulevard_peripherique:
    output_path = "output/detectors_matched_with_bvd_peripherique_2_osm_01_" + str(year)
else:
    output_path = "output/detectors_matched_2_osm_01_" + str(year)

In [11]:
# get OSM dataframe and process car detectors

if with_boulevard_peripherique:
    G_road_network = ox.graph_from_place("Paris", simplify=True, network_type="drive")
    df_detectors = df_car_detectors.copy()
    
else:
    alpha_shape = alphashape.alphashape(polygon_road_network, 435)
    coordinates = list(alpha_shape.exterior[0].coords)
    polygon = Polygon(coordinates)
    x_coords, y_coords = zip(*coordinates)

    overpass_settings = '[out:json][timeout:90][date:"' + str(year) + '-01-01T00:00:00Z"]'
    ox.settings.overpass_settings = overpass_settings
    ox.settings.log_console = True
    G_road_network = ox.graph_from_polygon(
        polygon, simplify=True, network_type="drive")
    
    boundary_gdf = gpd.GeoDataFrame(
    geometry=[polygon], crs=df_car_detectors.crs)
    df_car_detectors_without_dupl = df_car_detectors.drop_duplicates(
        subset='iu_ac', keep='first')
    car_detectors_within_boundary = gpd.sjoin(
        df_car_detectors_without_dupl, boundary_gdf, op='within')
    df_detectors = car_detectors_within_boundary.copy()
    
nodes_osm, df_osm = momepy.nx_to_gdf(G_road_network, points=True, lines=True)

/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_1537/2175760102.py:27: UserWarning: Approach is not set. Defaulting to 'primal'.
  nodes_osm, df_osm = momepy.nx_to_gdf(G_road_network, points=True, lines=True)


In [12]:
df_osm.head()

# Check for NaN values in all columns of df_osm
nan_percentages_all_columns = df_osm.isna().mean() * 100
print(nan_percentages_all_columns)




osmid          0.000000
lanes         53.390801
name           2.640624
highway        0.000000
maxspeed       1.031153
oneway         0.000000
reversed       0.000000
length         0.000000
geometry      10.966228
junction      97.130231
width         93.643952
bridge        98.554204
tunnel        98.766981
access        99.514431
ref           99.541710
node_start     0.000000
node_end       0.000000
dtype: float64


In [13]:
# Print rows where "geometry" is NaN
nan_geometry_rows = df_osm[df_osm['geometry'].isna()]
print(nan_geometry_rows)


            osmid lanes                      name       highway maxspeed  \
0        31093720     2         Rue Louis Lumière   residential       30   
55      720196717     2  Avenue Théophile Gautier      tertiary       30   
64       34172802     4          Boulevard Suchet       primary       50   
72       24981503     2  Allée des Fortifications  unclassified       30   
74     1295716614     2    Avenue de l'Hippodrome     secondary       50   
...           ...   ...                       ...           ...      ...   
18311    31093720     2         Rue Louis Lumière   residential       30   
18312  1211815123     2                Rue Érasme   residential       30   
18322   677991574     3        Quai André Citroën  primary_link       30   
18326    24663180     6      Avenue de Versailles       primary       30   
18327    24663180     6      Avenue de Versailles       primary       30   

       oneway reversed  length geometry junction width bridge tunnel access  \
0       

In [14]:
# Process osm network
df_osm['osm_id'] = range(1, len(df_osm) + 1)
df_osm.drop(columns=['width', 'bridge', 'tunnel', 'junction', 'access', 'ref'])

# Filter osm network for higher order roads
df_osm_hor = df_osm[
    df_osm["highway"].str.contains("motorway") |
    df_osm["highway"].str.contains("trunk") |
    df_osm["highway"].str.contains("primary") |
    df_osm["highway"].str.contains("secondary") |
    df_osm["highway"].str.contains("tertiary") 
]
df_osm_hor = df_osm_hor[df_osm_hor['geometry'].notnull()]

In [15]:
# Configure matching process

# Trade-off between scoring angle difference and centroid distance. alpha = 0.8 seemed to get the best matching. Keep in mind that we normalize the absolute difference of centroid 
# distance and angle difference is done in the score computation. 
alpha = 0.8

# Maximum centroid distance between two candidates
maximum_distance = 50

# Maximum angle difference between two candidates
maximum_angle = 15 * np.pi / 180.0

## Plot data

In [16]:
# fig, ax = plt.subplots()
# df_detectors.plot(ax=ax)

In [17]:
# fig, ax = plt.subplots()
# df_osm.plot(ax=ax)

In [18]:
# fig, ax = plt.subplots()
# df_osm_hor.plot(ax=ax)

## Matching

In [19]:
# Calculate centroids
detector_centroids = np.vstack([
    df_detectors["geometry"].centroid.x, df_detectors["geometry"].centroid.y]).T

osm_centroids = np.vstack([
    df_osm_hor["geometry"].centroid.x, df_osm_hor["geometry"].centroid.y]).T

/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_1537/3904791298.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_detectors["geometry"].centroid.x, df_detectors["geometry"].centroid.y]).T
/var/folders/m_/fjnjc1sn0ggc7z_2y7n27xfh0000gn/T/ipykernel_1537/3904791298.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df_osm_hor["geometry"].centroid.x, df_osm_hor["geometry"].centroid.y]).T


In [20]:
# Calculate orientation

def angle(geometry):
    coordinates = np.array(geometry.xy).T
    return np.arctan2(coordinates[-1, 1] - coordinates[0, 1], coordinates[-1, 0] - coordinates[0, 0])
    
detector_angles = df_detectors["geometry"].apply(angle).values
osm_angles = df_osm_hor["geometry"].apply(angle).values

In [21]:
# Calculate n-to-m distances
centroid_distances = np.zeros((len(detector_centroids), len(osm_centroids)))
angle_distances = np.zeros((len(detector_centroids), len(osm_centroids)))

In [22]:
for k in tqdm(range(len(detector_centroids))):
    centroid_distances[k,:] = la.norm(detector_centroids[k] - osm_centroids, axis = 1)
    angle_distances[k,:] = np.abs(detector_angles[k] - osm_angles)

angle_distances[angle_distances < 0.0] += 2.0 * np.pi

  0%|          | 0/3739 [00:00<?, ?it/s]

In [23]:
# Prepare scoring / matching
normalization_variable = angle_distances.mean()/centroid_distances.mean()

scores = alpha * centroid_distances + (1-alpha) * (1/normalization_variable) * angle_distances
# scores = alpha * centroid_distances + (1-alpha) * 0.1 * angle_distances

# Deactivate improbable matchings
scores[centroid_distances > maximum_distance] = np.inf
scores[angle_distances > maximum_angle] = np.inf

In [24]:
centroid_distances.mean()

0.0596865587245606

In [25]:
# Matching process
matchings = []
matched_scores = []

# The idea is relatively simple:
# - Find the matching with the smallest score among those with a finite value
# - Note down the matching, and set all matching containing the two links to Inf
# - Continue until no scores with finite value are left

current = np.count_nonzero(~np.isfinite(scores))

with tqdm(total = np.prod(scores.shape) - current) as progress:
    while np.count_nonzero(np.isfinite(scores)) > 0:
        # Find best score and note down
        index = np.unravel_index(np.argmin(scores), scores.shape)
        matched_scores.append(scores[index])

        # Set both invlved links to Inf
        scores[index[0], :] = np.inf
        scores[:, index[1]] = np.inf
        
        # Manage progress plotting
        update = np.count_nonzero(~np.isfinite(scores))
        
        if update > current:
            progress.update(update - current)
            current = update

        matchings.append(index)
        
matchings = np.array(matchings) # The matchings themselves (index reference, index matsim)
matched_scores = np.array(matched_scores) # The scores of the matchings

  0%|          | 0/2489134 [00:00<?, ?it/s]

## Output

In [26]:
# Construct a data set containing all matching information
df_matching = pd.DataFrame({
    "iu_ac": df_detectors.iloc[matchings[:, 0]]["iu_ac"].values,
    "geometry_detector": df_detectors.iloc[matchings[:, 0]]["geometry"].values,
    "osm_id": df_osm_hor.iloc[matchings[:,1]]["osm_id"].values,
    "lanes": df_osm_hor.iloc[matchings[:, 1]]["lanes"].values,
    "highway": df_osm_hor.iloc[matchings[:, 1]]["highway"].values,
    "oneway": df_osm_hor.iloc[matchings[:, 1]]["oneway"].values,
    "length_mapped_osm_street": df_osm_hor.iloc[matchings[:, 1]]["length"].values,
    "score": matched_scores,
    "date_start": df_detectors.iloc[matchings[:,0]]["date_debut"].values,
    "date_end": df_detectors.iloc[matchings[:,0]]["date_fin"].values,
})
df_matching = df_matching.sort_values(by='iu_ac')

In [27]:
df_matching['length_detector_street'] = df_matching['geometry_detector'].apply(lambda x: line_length_in_meters(x))
df_matching['lanes_mapped'] = df_matching['lanes'].apply(parse_and_average_lanes)
df_matched_with_lanes_approximated = approximate_number_of_lanes(df_matching)

In [28]:
if year == 2023:
    # nan_count = df_matched_with_lanes_approximated['lanes_mapped'].isna().sum()
    # print("Number of NaN values in 'lanes_mapped' column:", nan_count)
    
    # number of lanes are missing for highways of type "primary_link", "secondary_link", "trunk_link" and "tertiary_link"
    number_of_lanes_primary_link = df_matched_with_lanes_approximated[df_matched_with_lanes_approximated['highway'] == 'primary']['lanes_mapped'].mean()
    number_of_lanes_secondary_link = df_matched_with_lanes_approximated[df_matched_with_lanes_approximated['highway'] == 'secondary']['lanes_mapped'].mean()
    number_of_lanes_tertiary_link = df_matched_with_lanes_approximated[df_matched_with_lanes_approximated['highway'] == 'tertiary']['lanes_mapped'].mean()
    number_of_lanes_trunk_link = df_matched_with_lanes_approximated[df_matched_with_lanes_approximated['highway'] == 'trunk']['lanes_mapped'].mean()
    print(number_of_lanes_primary_link, number_of_lanes_secondary_link, number_of_lanes_trunk_link)
    
    df_matched_with_lanes_approximated.loc[df_matched_with_lanes_approximated['highway'] == 'primary_link', 'lanes_mapped'] = number_of_lanes_primary_link
    df_matched_with_lanes_approximated.loc[df_matched_with_lanes_approximated['highway'] == 'secondary_link', 'lanes_mapped'] = number_of_lanes_secondary_link
    df_matched_with_lanes_approximated.loc[df_matched_with_lanes_approximated['highway'] == 'tertiary_link', 'lanes_mapped'] = number_of_lanes_tertiary_link
    df_matched_with_lanes_approximated.loc[df_matched_with_lanes_approximated['highway'] == 'trunk_link', 'lanes_mapped'] = number_of_lanes_trunk_link
    
    # nan_count = df_matched_with_lanes_approximated['lanes_mapped'].isna().sum()
    # print("Number of NaN values in 'lanes_mapped' column:", nan_count)

In [29]:
p = 0.9
percentile = df_matched_with_lanes_approximated['score'].quantile(p)

df_matching_best = df_matched_with_lanes_approximated[df_matched_with_lanes_approximated['score'] < percentile]
gdf_matched = gpd.GeoDataFrame(df_matching_best, geometry='geometry_detector')

In [30]:
gdf_matched

,iu_ac,geometry_detector,osm_id,lanes,highway,oneway,length_mapped_osm_street,score,date_start,date_end,length_detector_street,lanes_mapped
2838,1,"LINESTRING (2.33342 48.86003, 2.33506 48.85965)",13594,"[5, 4]",primary,True,699.889,0.001053,1996-10-03 02:00:00,2023-01-01 01:00:00,127.877045,4.500000
1322,10,"LINESTRING (2.34947 48.85648, 2.35059 48.85606)",996,4,primary,True,148.501,0.000260,1996-10-03 02:00:00,2023-01-01 01:00:00,94.716807,4.000000
1031,100,"LINESTRING (2.34994 48.84888, 2.34916 48.84992)",3046,2,secondary,False,119.990,0.000200,1996-10-03 02:00:00,2023-01-01 01:00:00,128.196599,2.000000
2783,1001,"LINESTRING (2.40343 48.82928, 2.40184 48.82969...",16925,2,tertiary,True,54.790,0.000953,1996-10-03 02:00:00,2023-01-01 01:00:00,202.462897,2.000000
3332,1005,"LINESTRING (2.33249 48.82783, 2.32726 48.82788...",17191,NaN,primary,True,15.882,0.003538,1996-10-03 02:00:00,2023-01-01 01:00:00,418.329528,3.039246
...,...,...,...,...,...,...,...,...,...,...,...,...
925,994,"LINESTRING (2.39934 48.83079, 2.39797 48.83170)",12714,2,primary,False,117.089,0.000181,1996-10-03 02:00:00,2023-01-01 01:00:00,142.898625,2.000000
652,994,"LINESTRING (2.40099 48.82974, 2.39934 48.83079)",17978,2,primary,False,148.970,0.000135,2005-01-01 01:00:00,2019-06-01 02:00:00,167.555671,2.000000
3014,995,"LINESTRING (2.40087 48.82964, 2.40158 48.82919)",17979,1,primary,True,18.987,0.001431,1996-10-03 02:00:00,2023-01-01 01:00:00,72.996578,1.000000
2484,996,"LINESTRING (2.40170 48.82925, 2.40099 48.82974)",17681,2,primary,True,27.162,0.000649,1996-10-03 02:00:00,2023-01-01 01:00:00,75.908464,2.000000


In [31]:
detectors_delivering_data_2023 = np.loadtxt(data_path + 'detectors_delivering_data_2023.csv', delimiter=",", dtype=int)

In [32]:
detectors_delivering_data_2023

array([   5,    7,    8, ..., 7069, 7250, 7257])

In [33]:
# Convert 'iu_ac' column in gdf_matched to integers
gdf_matched['iu_ac'] = gdf_matched['iu_ac'].astype(int)
detectors_delivering_data_matched = gdf_matched[gdf_matched['iu_ac'].isin(detectors_delivering_data_2023)]

In [34]:
# Compute the total length of 'length_mapped_osm_street' for detectors_delivering_data_matched
total_length_mapped_osm_street = detectors_delivering_data_matched['length_mapped_osm_street'].sum()/1000

# Print the total length
print("Total length of 'length_mapped_osm_street' for detectors_delivering_data_matched:", total_length_mapped_osm_street)


Total length of 'length_mapped_osm_street' for detectors_delivering_data_matched: 240.65728000000001


### Detectors delivering data

In [56]:
# Group detectors_delivering_data_matched by 'highway' and create separate dataframes for each group
highway_groups = detectors_delivering_data_matched.groupby('highway')

# Create a dictionary to store the separate dataframes
highway_dfs = {highway: group for highway, group in highway_groups}

# Example: Accessing the dataframe for a specific highway type
primary_highway_df = highway_dfs.get('primary', pd.DataFrame())
secondary_highway_df = highway_dfs.get('secondary', pd.DataFrame())
tertiary_highway_df = highway_dfs.get('tertiary', pd.DataFrame())
trunk_highway_df = highway_dfs.get('trunk', pd.DataFrame())

# Group motorway and motorway_link together, and all other pairs to yield one length
grouped_highways = {
    'motorway': ['motorway', 'motorway_link'],
    'primary': ['primary', 'primary_link'],
    'secondary': ['secondary', 'secondary_link'],
    'tertiary': ['tertiary', 'tertiary_link'],
    'trunk': ['trunk', 'trunk_link']
}
    
# Convert the lengths from meters to kilometers for each highway type dataframe
primary_highway_df['length_detector_street_km'] = primary_highway_df['length_mapped_osm_street'] / 1000
secondary_highway_df['length_detector_street_km'] = secondary_highway_df['length_mapped_osm_street'] / 1000
tertiary_highway_df['length_detector_street_km'] = tertiary_highway_df['length_mapped_osm_street'] / 1000
trunk_highway_df['length_detector_street_km'] = trunk_highway_df['length_mapped_osm_street'] / 1000

# Print the total lengths in kilometers for each highway type
print("Primary highway detectors total length (km):", primary_highway_df['length_detector_street_km'].sum())
print("Secondary highway detectors total length (km):", secondary_highway_df['length_detector_street_km'].sum())
print("Tertiary highway detectors total length (km):", tertiary_highway_df['length_detector_street_km'].sum())
print("Trunk highway detectors total length (km):", trunk_highway_df['length_detector_street_km'].sum())

# Calculate the percentage of each highway type length in comparison to the total network length
primary_percentage = (primary_highway_df['length_detector_street_km'].sum() / total_network_length_km) * 100
secondary_percentage = (secondary_highway_df['length_detector_street_km'].sum() / total_network_length_km) * 100
tertiary_percentage = (tertiary_highway_df['length_detector_street_km'].sum() / total_network_length_km) * 100
trunk_percentage = (trunk_highway_df['length_detector_street_km'].sum() / total_network_length_km) * 100

# Print the percentages for each highway type
print(f"Primary highway detectors percentage of total network: {primary_percentage:.2f}%")
print(f"Secondary highway detectors percentage of total network: {secondary_percentage:.2f}%")
print(f"Tertiary highway detectors percentage of total network: {tertiary_percentage:.2f}%")
print(f"Trunk highway detectors percentage of total network: {trunk_percentage:.2f}%")

# Create a dataframe for the 'motorway' highway type
motorway_highway_df = highway_dfs.get('motorway', pd.DataFrame())

# Convert the lengths from meters to kilometers for the motorway highway type dataframe
motorway_highway_df['length_detector_street_km'] = motorway_highway_df['length_mapped_osm_street'] / 1000

# Print the total length in kilometers for the motorway highway type
print("Motorway highway detectors total length (km):", motorway_highway_df['length_detector_street_km'].sum())

# Calculate the percentage of motorway highway type length in comparison to the total network length
motorway_percentage = (motorway_highway_df['length_detector_street_km'].sum() / total_network_length_km) * 100

# Print the percentage for the motorway highway type
print(f"Motorway highway detectors percentage of total network: {motorway_percentage:.2f}%")



Primary highway detectors total length (km): 116.990911
Secondary highway detectors total length (km): 52.36389200000001
Tertiary highway detectors total length (km): 14.527676000000001
Trunk highway detectors total length (km): 49.076685
Primary highway detectors percentage of total network: 6.52%
Secondary highway detectors percentage of total network: 2.92%
Tertiary highway detectors percentage of total network: 0.81%
Trunk highway detectors percentage of total network: 2.74%
Motorway highway detectors total length (km): 0.918958
Motorway highway detectors percentage of total network: 0.05%


In [36]:
# Calculate the total length of highways with classification "Trunk", "Motorway", "Primary", "Secondary", and "Tertiary" in df_osm
highway_types = ['trunk', 'motorway', 'primary', 'secondary', 'tertiary']
highway_lengths = {}

for highway in highway_types:
    highway_lengths[highway] = df_osm[df_osm['highway'].isin([highway, f'{highway}_link'])]['length'].sum()

# Calculate the total length of the entire network in df_osm
total_network_length = df_osm['length'].sum()

# Convert the lengths from meters to kilometers
for highway in highway_lengths:
    highway_lengths[highway] = highway_lengths[highway] / 1000

total_network_length_km = total_network_length / 1000
print(f"Total network length: {total_network_length_km:.2f} km")


# Calculate the percentage of each highway type length in comparison to the total network length
highway_percentages = {highway: (length / total_network_length_km) * 100 for highway, length in highway_lengths.items()}

# Print the results in kilometers and percentages
for highway in highway_lengths:
    print(f"Total length of {highway} highways: {highway_lengths[highway]:.2f} km")
    print(f"Percentage of {highway} highways in the total network: {highway_percentages[highway]:.2f}%")

Total network length: 1793.19 km
Total length of trunk highways: 108.84 km
Percentage of trunk highways in the total network: 6.07%
Total length of motorway highways: 5.63 km
Percentage of motorway highways in the total network: 0.31%
Total length of primary highways: 295.57 km
Percentage of primary highways in the total network: 16.48%
Total length of secondary highways: 218.67 km
Percentage of secondary highways in the total network: 12.19%
Total length of tertiary highways: 228.15 km
Percentage of tertiary highways in the total network: 12.72%


In [37]:
# save the results

# df_matching_best.to_csv(output_path + "_best_matches.csv", sep=";", index=False)
# df_matched_with_lanes_approximated.to_csv(output_path + ".csv", sep=";", index=False)

## Plot for districts

In [38]:
# districts = gpd.read_file(data_path + 'districts_paris.geojson')

In [39]:
# # filter detectors for date_debut and date_fin
# start_date = pd.Timestamp('2023-01-01 00:00:00+0000', tz='UTC')
# df_detectors_2023 = df_detectors[(df_detectors['date_debut'] <= start_date) & (df_detectors['date_fin'] >= start_date)]

# zone1 = districts[(districts['c_ar'] == 1) | (districts['c_ar'] == 2) | (districts['c_ar'] == 3) | (districts['c_ar'] == 4)]
# zone2 = districts[(districts['c_ar'] == 5) | (districts['c_ar'] == 6) | (districts['c_ar'] == 7)]
# zone_1_boundary = zone1.geometry.unary_union
# zone_2_boundary = zone2.geometry.unary_union

# gdf_matched_intersect_zone_1_boundary = gdf_matched[gdf_matched.intersects(zone_1_boundary)]
# df_osm_hor_intersect_zone_1_boundary = df_osm_hor[df_osm_hor.intersects(zone_1_boundary)]
# df_detectors_intersect_zone_1_boundary = df_detectors_2023[df_detectors_2023.intersects(zone_1_boundary)]

# gdf_matched_intersect_zone_2_boundary = gdf_matched[gdf_matched.intersects(zone_2_boundary)]
# df_osm_hor_intersect_zone_2_boundary = df_osm_hor[df_osm_hor.intersects(zone_2_boundary)]
# df_detectors_intersect_zone_2_boundary = df_detectors_2023[df_detectors_2023.intersects(zone_2_boundary)]
# df_osm_intersect_zone_1_boundary = df_osm[df_osm.intersects(zone_1_boundary)]
# df_osm_intersect_zone_2_boundary = df_osm[df_osm.intersects(zone_2_boundary)]

In [40]:
# fig, ax = plt.subplots(figsize=(12, 10))
# df_osm_intersect_zone_1_boundary.plot(ax=ax, linewidth = 0.5, color = "grey", label = "OSM road network")
# df_osm_hor_intersect_zone_1_boundary.plot(ax=ax, color = 'orange', linewidth = 4, label = "OSM higher order roads on 01.01.2024")
# df_detectors_intersect_zone_1_boundary.plot(ax=ax, linewidth=2, color = "green", label = "Detector network")
# gdf_matched_intersect_zone_1_boundary.plot(ax=ax, color = 'blue', linewidth=1, label = "Detectors matched to OSM higher order roads")

# df_osm_intersect_zone_2_boundary.plot(ax=ax, linewidth = 0.5, color = "grey")
# df_osm_hor_intersect_zone_2_boundary.plot(ax=ax, color = 'orange', linewidth = 4)
# df_detectors_intersect_zone_2_boundary.plot(ax=ax, linewidth=2, color = "green")
# gdf_matched_intersect_zone_2_boundary.plot(ax=ax, color = 'blue', linewidth=1)

# exterior_x, exterior_y = zone_1_boundary.exterior.xy
# plt.plot(exterior_x, exterior_y, color = 'magenta', linewidth = 4, label = "District 1 boundary")

# exterior_x_zone_2, exterior_y_zone_2 = zone_2_boundary.exterior.xy
# plt.plot(exterior_x_zone_2, exterior_y_zone_2, color = 'purple', linewidth = 4, label = "District 2 boundary")

# plt.xlabel("Longitude", font = font, fontsize = fontsize)
# plt.ylabel("Latitude", font = font, fontsize = fontsize)
# plt.title("Zone 1 and 2: Matches of detectors to OSM higher order roads", font = font, fontsize = fontsize)
# plt.xticks(font = font, fontsize = 15)
# plt.yticks(font = font, fontsize = 15)
# font_legend = font_manager.FontProperties(family=font, style='normal', size=15)
# plt.legend(loc='lower left', prop = font_legend)
# plt.savefig("results/zone_1_matched.pdf", dpi=1000, bbox_inches='tight')

In [41]:
# fig, ax = plt.subplots(figsize=(12, 10))
# df_osm_intersect_zone_1_boundary.plot(ax=ax, linewidth = 0.5, color = "grey", label = "OSM road network")
# df_osm_hor_intersect_zone_1_boundary.plot(ax=ax, color = 'orange', linewidth = 4, label = "OSM higher order roads on 01.01.2024")
# df_detectors_intersect_zone_1_boundary.plot(ax=ax, linewidth=2, color = "green", label = "Detector network")

# df_osm_intersect_zone_2_boundary.plot(ax=ax, linewidth = 0.5, color = "grey")
# df_osm_hor_intersect_zone_2_boundary.plot(ax=ax, color = 'orange', linewidth = 4)
# df_detectors_intersect_zone_2_boundary.plot(ax=ax, linewidth=2, color = "green")

# exterior_x, exterior_y = zone_1_boundary.exterior.xy
# plt.plot(exterior_x, exterior_y, color = 'magenta', linewidth = 4, label = "District 1 boundary")

# exterior_x_zone_2, exterior_y_zone_2 = zone_2_boundary.exterior.xy
# plt.plot(exterior_x_zone_2, exterior_y_zone_2, color = 'purple', linewidth = 4, label = "District 2 boundary")

# plt.xlabel("Longitude", font = font, fontsize = fontsize)
# plt.ylabel("Latitude", font = font, fontsize = fontsize)
# plt.title("Zone 1 and 2: OSM higher roads and detector network", font = font, fontsize = fontsize)
# plt.xticks(font = font, fontsize = 15)
# plt.yticks(font = font, fontsize = 15)
# font_legend = font_manager.FontProperties(family=font, style='normal', size=15)
# plt.legend(loc='lower left', prop = font_legend)
# plt.savefig("results/zone_1_osm_hor_and_dets.pdf", dpi=1000, bbox_inches='tight')

In [42]:
# def get_length_in_lane_km(gdf):
#     length_in_lane_km = 0

#     for idx, row in gdf.iterrows():
#         length = row['length_detector_street']
#         lanes = row['lanes_mapped']
#         if pd.isna(lanes):
#             continue
#         if pd.isna(length):
#             continue
#         length_in_lane_km += length * lanes / 1000
#         # print(length_in_lane_km)
#     return length_in_lane_km

# length_in_lane_km_zone_1 = get_length_in_lane_km(gdf_matched_intersect_zone_1_boundary)
# length_in_lane_km_zone_2 = get_length_in_lane_km(gdf_matched_intersect_zone_2_boundary)
    
# print(length_in_lane_km_zone_1)
# print(length_in_lane_km_zone_2)    

We find, for 01.01.2024:

Z1: 78.66487745107128
Z2: 121.94722011749302

## Plot overall

In [43]:
# fig, ax = plt.subplots(figsize=(12, 10))
# # df_matching_best.plot(ax=ax, color = 'magenta', linewidth=1, label = "Matches")
# df_osm_hor.plot(ax=ax, color = 'orange', linewidth = 4, label = "OSM higher order roads")
# df_detectors.plot(ax=ax, linewidth=1, color = "green", label = "Detectors")

# plt.xlabel("Longitude", font = font, fontsize = fontsize)
# plt.ylabel("Latitude", font = font, fontsize = fontsize)
# plt.title("Paris - OSM higher order roads and detector matches", font = font, fontsize = fontsize)

# plt.xticks(font = font, fontsize = fontsize)
# plt.yticks(font = font, fontsize = fontsize)
# font_legend = font_manager.FontProperties(family=font, style='normal', size=15)
# plt.legend(loc='upper left', prop = font_legend)

# plt.savefig("results/osm_hor_and_detectors.pdf", dpi=1000)

In [44]:
# df_comparison = df_detectors.copy()
# df_comparison = pd.merge(df_comparison, df_matching)
# df_comparison.to_file("output/detectors_matched_2_osm.geojson", driver = 'GeoJSON')

In [45]:
# osm_matched_2_detectors = df_osm_hor.copy()
# osm_matched_2_detectors = pd.merge(osm_matched_2_detectors, df_matching)
# osm_matched_2_detectors.to_file(
#     "output/osm_matched_2_detectors.geojson", driver='GeoJSON')